In [1]:
import pandas as pd
import numpy as np
import random

df_punk=pd.read_csv ("data/punk-4101.csv")
df_transaction=pd.read_csv ("data/transaction-4101.csv")

c:\users\natas\appdata\local\programs\python\python39\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [2]:
df_punk.drop(columns=["Unnamed: 0"],inplace=True)
df_punk.rename(columns={'_id': 'id_punk','_type': 'type'}, inplace=True)

In [3]:
df_transaction.drop(columns=["Unnamed: 0"],inplace=True)

In [4]:
def put_id(col, col_id,df):
    name_id=col.lower()
    col_dict= {name_id:list(df[col].unique()),
         col_id:list(range(1,len(df[col].unique())+1))}
    df[col_id]=df[col].apply(lambda x:col_dict.get(col_id)[col_dict.get(name_id).index(x)])
    
    df_col=pd.DataFrame(col_dict) #to return!
    print(col)
    print(df_col.head())
    print("**********************************************************")
    return df_col

In [5]:
#punk
df_type=put_id("type","id_type",df_punk)
df_attr=put_id("attr","id_attr",df_punk)

df_punk_id=df_punk[["id_punk","id_type"]].groupby("id_punk").first().reset_index()
df_punk_attr=df_punk[["id_punk","id_attr"]]

type
     type  id_type
0  Female        1
1    Male        2
2  Zombie        3
3     Ape        4
4   Alien        5
**********************************************************
attr
               attr  id_attr
0  Green Eye Shadow        1
1           Earring        2
2        Blonde Bob        3
3             Smile        4
4            Mohawk        5
**********************************************************


In [7]:
df_tr_user=pd.concat([df_transaction["from"],df_transaction["to"]],ignore_index=True)
df_tr_user=df_tr_user.sort_values().unique()
##.dropna()
#df_tr_user=df_tr_user.sort_values(unique()
#df_transaction[['from','to']].fillna('',inplace=True)

In [8]:
df_transaction["id_from"]=''
df_transaction["id_to"]=''
col_dict_user= {'user':list(df_tr_user),
         'id_user':list(range(1,len(df_tr_user)+1))}
df_transaction["id_from"][df_transaction['from'].notna()]=df_transaction["from"].apply(lambda x: '' if x=='NaN' else col_dict_user.get("id_user")[col_dict_user.get('user').index(x)])
df_transaction["id_to"][df_transaction['to'].notna()]=df_transaction["to"].apply(lambda x: '' if x=='NaN' else col_dict_user.get("id_user")[col_dict_user.get('user').index(x)])
df_user=pd.DataFrame(col_dict_user)[:-1] #to return!

<ipython-input-8-8c52087f3a36>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_transaction["id_from"][df_transaction['from'].notna()]=df_transaction["from"].apply(lambda x: '' if x=='NaN' else col_dict_user.get("id_user")[col_dict_user.get('user').index(x)])
<ipython-input-8-8c52087f3a36>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_transaction["id_to"][df_transaction['to'].notna()]=df_transaction["to"].apply(lambda x: '' if x=='NaN' else col_dict_user.get("id_user")[col_dict_user.get('user').index(x)])


In [9]:
df_trans=df_transaction[['id_punk','amount_dol','amount_ethereums', 'letter', 'date', 'id_type_transaction', 'id_from',
       'id_to']]
df_trans['date']=pd.to_datetime(df_trans['date'])

<ipython-input-9-29cdceeb6991>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trans['date']=pd.to_datetime(df_trans['date'])


In [38]:
from sqlalchemy import create_engine
from sqlalchemy import text
import pandas as pd
from datetime import date


host = "127.0.0.1:3306"
user="root"
passwd="ns3166"
database="punk"


engine = create_engine('mysql+mysqlconnector://' + user + ':' + passwd + '@' + host + '/' + database)

df_type.to_sql("type",con=engine, if_exists="append", index=False)
df_user.to_sql("users",con=engine, if_exists="append", index=False)
df_attr.to_sql("attributts",con=engine, if_exists="append", index=False)
df_punk_id.to_sql("punk",con=engine, if_exists="append", index=False)
df_punk_attr.to_sql("punk_attributts",con=engine, if_exists="append", index=False)
df_type_transaction.to_sql("type_transaction",con=engine, if_exists="append", index=False)



In [46]:
df_trans[1:2].to_sql("transaction",con=engine, if_exists="append", index=False)

DatabaseError: (mysql.connector.errors.DatabaseError) 1366 (HY000): Incorrect integer value: '' for column 'id_to' at row 1
[SQL: INSERT INTO transaction (id_punk, amount_dol, amount_ethereums, letter, date, id_type_transaction, id_from, id_to) VALUES (%(id_punk)s, %(amount_dol)s, %(amount_ethereums)s, %(letter)s, %(date)s, %(id_type_transaction)s, %(id_from)s, %(id_to)s)]
[parameters: {'id_punk': 0.0, 'amount_dol': '1.26', 'amount_ethereums': '321', 'letter': 'M', 'date': datetime.datetime(2021, 9, 4, 0, 0), 'id_type_transaction': 2, 'id_from': 5440, 'id_to': ''}]
(Background on this error at: https://sqlalche.me/e/14/4xp6)

In [20]:
df_attr

,attr,id_attr
0,Green Eye Shadow,1
1,Earring,2
2,Blonde Bob,3
3,Smile,4
4,Mohawk,5
...,...,...
82,Orange Side,83
83,Wild Blonde,84
84,Buck Teeth,85
85,Top Hat,86


In [27]:
df_user

,user,id_user
0,0x0000000000000000000000000000000000000001,1
1,0x0000000000000000000000000000000000000004,2
2,0x0000000000000000000000000000000000000011,3
3,0x0000000000000000000000000000000000000014,4
4,0x0000000000000000000000000000000000000031,5
...,...,...
6005,0xffe6788be411c4353b3b2c546d0401d4d8b2b3ed,6006
6006,0xfff1b1e92accedfd67b01b1058edb123cbbdede7,6007
6007,0xfff423212eb0b97788d43a0c38b7d5762ba3c6e6,6008
6008,0xfff7b45fea022ea9bb5822f5a065e31d8e235b0d,6009


In [28]:
df_type.to_csv(r"C:\Users\natas\Downloads\type.csv")
df_user.to_csv(r"C:\Users\natas\Downloads\users.csv")
df_attr.to_csv(r"C:\Users\natas\Downloads\attr.csv")
df_type_transaction.to_csv(r"C:\Users\natas\Downloads\type_transaction.csv")

In [29]:
df_trans

,id_punk,amount_dol,amount_ethereums,letter,date,id_type_transaction,id_from,id_to
0,0.0,"4,048",1,NaN,2021-12-25,1,1149,
1,0.0,1.26,321,M,2021-09-04,2,5440,
2,0.0,1.14,321,M,2021-09-01,1,5440,
3,0.0,1.11,320,M,2021-09-01,2,1142,
4,0.0,1.1,320,M,2021-09-01,1,1142,
...,...,...,...,...,...,...,...,...
70486,4101.0,"9,517",8.0,NaN,2021-01-14,4,,
70487,4101.0,NaN,NaN,NaN,2021-01-08,7,,
70488,4101.0,"6,748",20.0,NaN,2020-10-08,4,,
70489,4101.0,NaN,NaN,NaN,2017-06-26,5,4566,3284
